In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
%matplotlib inline

In [3]:
data = pd.read_csv('~/Desktop/project_data_new/embedding_768_TCGA_COAD.csv', header=None)

In [4]:
data

,0,1,2,3,4,5,6,7,8,9,...,760,761,762,763,764,765,766,767,768,769
0,0.000000,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.000000,...,760.000000,761.000000,762.000000,763.000000,764.000000,765.000000,766.000000,767.000000,ID,PatientID
1,-0.457220,0.851885,0.540813,-0.423622,0.714115,-0.564329,1.607602,0.543431,-0.870704,0.980508,...,0.272053,0.577840,-0.313309,0.463409,0.216033,1.875605,-0.811024,-1.250726,TCGA-A6-6650-01Z-00-DX1.92f39e59-8784-4dfd-a06...,TCGA-A6-6650
2,-0.387709,0.730046,0.720710,-0.723050,0.841878,-0.463233,1.442049,0.573482,-0.817522,1.146394,...,0.308657,0.696497,-0.164076,0.249183,0.075877,2.179679,-0.814272,-1.311477,TCGA-A6-6142-01Z-00-DX1.e923ce20-d3c3-4d21-9e7...,TCGA-A6-6142
3,-0.430112,0.766393,0.768542,-0.344121,0.560576,-0.597754,1.708495,0.419003,-0.934103,1.152953,...,0.399675,0.471158,-0.089670,0.701613,0.178599,1.802830,-0.844305,-1.300183,TCGA-5M-AAT4-01Z-00-DX1.725C46CA-9354-43AC-AA8...,TCGA-5M-AAT4
4,-0.417006,0.964753,0.525541,-0.572356,0.777732,-0.462124,1.476457,0.640170,-0.722137,1.066126,...,0.094860,0.659872,-0.243587,0.122108,0.107965,2.146043,-0.946326,-1.345658,TCGA-A6-2676-01Z-00-DX1.c465f6e0-b47c-48e9-bdb...,TCGA-A6-2676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,-0.359843,0.929950,0.543955,-0.350527,0.404216,-0.566966,1.688820,0.196331,-1.050643,1.162059,...,0.404179,0.325667,-0.166769,0.612408,0.197617,1.686868,-0.917694,-1.251037,TCGA-G4-6321-01Z-00-DX1.20bd4687-4b24-4666-a72...,TCGA-G4-6321
457,-0.480253,0.931134,0.455165,-0.714430,0.685353,-0.373306,1.511218,0.747226,-0.652520,0.907837,...,0.062442,0.249245,-0.573766,-0.159084,-0.157464,2.173445,-0.916892,-1.098334,TCGA-RU-A8FL-01A-01-TSA.3743238D-C8AE-49A3-B5C...,TCGA-RU-A8FL
458,-0.397405,0.793659,0.682602,-0.526544,0.733674,-0.456034,1.593480,0.500975,-0.982290,1.068642,...,0.324934,0.458768,-0.172873,0.498273,0.168238,1.823696,-0.805411,-1.238766,TCGA-G4-6303-01Z-00-DX1.5819f041-d9b8-4fd2-bf5...,TCGA-G4-6303
459,-0.422558,0.896717,0.599262,-0.347394,0.683292,-0.429929,1.659195,0.494966,-0.979689,1.184772,...,0.340869,0.324060,-0.257918,0.544318,0.087316,1.892267,-0.869365,-1.202163,TCGA-QG-A5Z1-01Z-00-DX2.2CE72B6A-557F-43BD-BA4...,TCGA-QG-A5Z1
